# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [46]:
#calculating the counts and averages
schoolsTot = school_data["School ID"].count()

studentsTot = student_data["Student ID"].count()

budgetTot = school_data["budget"].sum()

mathAvg = student_data["math_score"].mean()

readAvg = student_data["reading_score"].mean()

passRate = (mathAvg + readAvg)/2

#getting the students with passing marks for Math. 
only70Math = student_data.loc[student_data["math_score"] >= 70]
mathPassPerc = (only70Math["Student ID"].count()/studentsTot)*100

#same for reading
only70Read = student_data.loc[student_data["reading_score"] >= 70]
readPassPerc = (only70Read["Student ID"].count()/studentsTot)*100

#Creating a dataframe for the summary 
summaryDict = {"total schools": schoolsTot,"total students":studentsTot, "total budget":budgetTot,
               "average math score":mathAvg,"average reading score":readAvg, "overall passing rate":passRate ,
              "% of pass students in math":mathPassPerc,"% of pass students in reading":readPassPerc}
distSummary = pd.DataFrame(summaryDict,index=[0])

#formatting. Assiging to a new df in case I have use the unformatted one for further processing
distSummary_final = distSummary.style.format({"total students" : '{:,}',
                                             "total budget" : "${:,}"})
distSummary_final


,total schools,total students,total budget,average math score,average reading score,overall passing rate,% of pass students in math,% of pass students in reading
0,15,"39,170","$24,649,428",78.9854,81.8778,80.4316,74.9809,85.8055


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [47]:
groupedSchoolData = school_data_complete.groupby(["school_name"]).agg({"type": "first","student_name":'count',
                                                                       "budget":"first","math_score":'mean',
                                                                       "reading_score":'mean'})

groupedSchoolData["Per Student Budget"] = groupedSchoolData["budget"]/groupedSchoolData["student_name"]

perSchool70Math = only70Math.groupby(["school_name"]).agg({"student_name":'count'})
percMathPass = (perSchool70Math["student_name"]/groupedSchoolData["student_name"])*100
groupedSchoolData["% Passing Math"] = percMathPass

perSchool70Read = only70Read.groupby(["school_name"]).agg({"student_name":'count'})
percReadPass = (perSchool70Read["student_name"]/groupedSchoolData["student_name"])*100
groupedSchoolData["% Passing Reading"] = percReadPass

groupedSchoolData["% Overall passing rate"] = (groupedSchoolData["% Passing Math"]+groupedSchoolData["% Passing Reading"])/2

groupedSchoolData = groupedSchoolData.sort_values("% Overall passing rate",ascending=False)
groupedSchoolData.head()

#formatting
groupedSchoolData_final = groupedSchoolData.style.format({"budget" : '${:,}',
                                             "Per Student Budget" : "${:,}"})
groupedSchoolData_final = groupedSchoolData_final.rename(columns={"type":"School Type",
                                                                  "student_name" :"Total Students",
                                                                 "budget":"Total School Budget",
                                                                 "math_score":"Average Math Score",
                                                                 "reading_score":"Average Reading Score"})

groupedSchoolData_final
#groupedSchoolData_final


#need to rename columns

AttributeError: 'Styler' object has no attribute 'rename'

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
groupedSchoolData.tail()

,type,student_name,budget,math_score,reading_score,Per Student Budget,% Passing Math,% Passing Reading,Overall passing rate
school_name,,,,,,,,,
Ford High School,District,2739,1763916,77.102592,80.746258,644.0,68.309602,79.299014,73.804308
Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,66.057551,81.222432,73.639992
Huang High School,District,2917,1910635,76.629414,81.182722,655.0,65.683922,81.316421,73.500171
Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,65.988471,80.739234,73.363852
Rodriguez High School,District,3999,2547363,76.842711,80.744686,637.0,66.366592,80.220055,73.293323


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [9]:
grade9_df = student_data.loc[student_data["grade"] == "9th"]

avgMath_grade9 = grade9_df.groupby(["school_name"]).agg({"math_score":'mean'})
avgMath_grade9 = avgMath_grade9.rename(columns={"math_score":"9th"})


grade10_df = student_data.loc[student_data["grade"] == "10th"]
avgMath_grade10 = grade10_df.groupby(["school_name"]).agg({"math_score":'mean'})
avgMath_grade10 = avgMath_grade10.rename(columns={"math_score":"10th"})


grade11_df = student_data.loc[student_data["grade"] == "11th"]
avgMath_grade11 = grade11_df.groupby(["school_name"]).agg({"math_score":'mean'})
avgMath_grade11 = avgMath_grade11.rename(columns={"math_score":"11th"})


grade12_df = student_data.loc[student_data["grade"] == "12th"]
avgMath_grade12 = grade12_df.groupby(["school_name"]).agg({"math_score":'mean'})
avgMath_grade12 = avgMath_grade12.rename(columns={"math_score":"12th"})


mathByGrade = pd.concat([avgMath_grade9, avgMath_grade10,avgMath_grade11,avgMath_grade12], axis=1, sort=False)

mathByGrade


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:

avgRead_grade9 = grade9_df.groupby(["school_name"]).agg({"reading_score":'mean'})
avgRead_grade9 = avgRead_grade9.rename(columns={"reading_score":"9th"})


avgRead_grade10 = grade10_df.groupby(["school_name"]).agg({"reading_score":'mean'})
avgRead_grade10 = avgRead_grade10.rename(columns={"reading_score":"10th"})


avgRead_grade11 = grade11_df.groupby(["school_name"]).agg({"reading_score":'mean'})
avgRead_grade11 = avgRead_grade11.rename(columns={"reading_score":"11th"})


avgRead_grade12 = grade12_df.groupby(["school_name"]).agg({"reading_score":'mean'})
avgRead_grade12 = avgRead_grade12.rename(columns={"reading_score":"12th"})


ReadByGrade = pd.concat([avgRead_grade9, avgRead_grade10,avgRead_grade11,avgRead_grade12], axis=1, sort=False)

ReadByGrade


,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [11]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [12]:
groupedSchoolData["Math Passed - Count"] = perSchool70Math["student_name"]
groupedSchoolData["Reading Passed - Count"] = perSchool70Read["student_name"]
groupedSchoolData["Spending Category"] = pd.cut(groupedSchoolData["Per Student Budget"],spending_bins, labels=group_names)
spendSummary = groupedSchoolData.groupby(["Spending Category"]).agg({"math_score":'mean',"reading_score":'mean',
                                                                    "Math Passed - Count":'sum',
                                                                    "Reading Passed - Count":'sum',
                                                                    "student_name" : 'sum'})
spendSummary["% Passing Math"] = (spendSummary["Math Passed - Count"]/spendSummary["student_name"])*100
spendSummary["% Passing Reading"] = (spendSummary["Reading Passed - Count"]/spendSummary["student_name"])*100
spendSummary["Overall Passing Rate"] = (spendSummary["% Passing Math"] + spendSummary["% Passing Reading"])/2
#spendSummary_final = spendSummary[[m]]
spendSummary.head()

#trim the df to have remove counts, rename columns

,math_score,reading_score,Math Passed - Count,Reading Passed - Count,student_name,% Passing Math,% Passing Reading,Overall Passing Rate
Spending Category,,,,,,,,
<$585,83.455399,83.933814,5967,6157,6368,93.702889,96.686558,95.194724
$585-615,83.599686,83.885211,2563,2611,2723,94.124128,95.886889,95.005509
$615-645,79.079225,81.891436,12685,14855,17766,71.400428,83.614770,77.507599
$645-675,76.997210,81.027843,8155,9987,12313,66.230813,81.109397,73.670105


## Scores by School Size

* Perform the same operations as above, based on school size.

In [13]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [17]:
groupedSchoolData["School Size"] = pd.cut(groupedSchoolData["student_name"],size_bins, labels=group_names)
sizeSummary = groupedSchoolData.groupby(["School Size"]).agg({"math_score":'mean',"reading_score":'mean',
                                                                    "Math Passed - Count":'sum',
                                                                    "Reading Passed - Count":'sum',
                                                                    "student_name" : 'sum'})
sizeSummary["% Passing Math"] = (sizeSummary["Math Passed - Count"]/sizeSummary["student_name"])*100
sizeSummary["% Passing Reading"] = (sizeSummary["Reading Passed - Count"]/sizeSummary["student_name"])*100
sizeSummary["Overall Passing Rate"] = (sizeSummary["% Passing Math"] + sizeSummary["% Passing Reading"])/2
sizeSummary.head()

,math_score,reading_score,Math Passed - Count,Reading Passed - Count,student_name,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,,,,
Small (<1000),83.821598,83.929843,1305,1334,1389,93.952484,96.040317,94.996400
Medium (1000-2000),83.374684,83.864438,7978,8247,8522,93.616522,96.773058,95.194790
Large (2000-5000),77.746417,81.344493,20087,24029,29259,68.652380,82.125158,75.388769


## Scores by School Type

* Perform the same operations as above, based on school type.

In [24]:
schoolType = groupedSchoolData.groupby(["type"]).agg({"math_score":'mean',"reading_score":'mean',
                                                                    "Math Passed - Count":'sum',
                                                                    "Reading Passed - Count":'sum',
                                                                    "student_name" : 'sum'})
schoolType["% Passing Math"] = (schoolType["Math Passed - Count"]/schoolType["student_name"])*100
schoolType["% Passing Reading"] = (schoolType["Reading Passed - Count"]/schoolType["student_name"])*100
schoolType["Overall Passing Rate"] = (schoolType["% Passing Math"] + schoolType["% Passing Reading"])/2

schoolType_final = schoolType.drop(columns=["Math Passed - Count","Reading Passed - Count","student_name"])
schoolType_final

,math_score,reading_score,% Passing Math,% Passing Reading,Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,93.701821,96.645891,95.173856
District,76.956733,80.966636,66.518387,80.905249,73.711818


In [ ]:
#Junk catcher cell::::::::: Please don't execute::::::::::::::::

#school_data_complete.head()
df = pd.DataFrame([[1,1.23,'Hello']], columns=list('ABC'))
df
data = {'a' : 1, 'b' : 2, 'c' : 3}
pd.DataFrame(data, index=[0])
student_data["Avg of scores"] = (student_data["math_score"] + student_data["reading_score"])/2
student_data.head()
passRate2 = student_data["Avg of scores"].mean()


#groupedSchoolData = school_data_complete.groupby(["school_name"])
#groupedSchoolData.head()
#studentsTotPerSchool = groupedSchoolData["student_name"].count()
#studentsTotPerSchool.head()
#state_duration = grouped_usa_df["duration (seconds)"].sum()
#state_duration.head()

#groupedSchoolData = school_data_complete.groupby(["school_name"]).agg({"student_name":'count',"math_score":'mean',"reading_score":'mean',"student_name": 'count'})


groupedSchoolData["School Size"] = pd.cut(groupedSchoolData["student_name"],size_bins, labels=group_names)

groupedSchoolData.head()
sizeSummary = groupedSchoolData.groupby(["School Size"]).agg({"math_score":'mean',"reading_score":'mean',
                                                                    "Math Passed - Count":'sum',
                                                                    "Reading Passed - Count":'sum',
                                                                    "student_name" : 'sum'})
sizeSummary

schoolType_final = schoolType.drop(columns=["Math Passed - Count","Reading Passed - Count","student_name"])
#df.drop(columns=['B', 'C'])
schoolType_final

#cost_calc.style.format({'c.somecolumn.cost'       : "${:,.2f}",
 #                       'c.somecolumn.cost'       : "${:,.2f}",
  #                      'e.somecolumn.cost'       : "${:,.2f}",
   #                     'e.somecolumn.cost'       : "${:,.2f}"
#need to format